In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Hamering.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Pushing.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Pulling.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Slapping.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Headering.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/FrontKicking.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Elbowing.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Kneeing.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/SideKicking.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/log/Punching.log
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/txt/Sidekicking.txt
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/txt/Kneeing.txt
/kaggle/input/emg-physical-action-data-set/sub3/Aggressive/txt/Slapping.txt
/kagg

In [4]:

import glob
path = "/kaggle/input/emg-physical-action-data-set/sub/Normal/*.csv"
idx = 0
df_normal = pd.DataFrame()
for file in glob.glob(path):
    prev_df = pd.read_csv(file)
    # print(df.shape)
    df_normal = pd.concat([df_normal,prev_df],ignore_index=True)
    # print(df.shape)

In [5]:
df_normal.head()

,0,1,2,3,4,5,6,7,label
0,874.50,27.75,25.50,-84.00,306.75,15.75,-4.25,-4.50,2
1,346.75,24.25,-4.25,-110.00,284.00,26.25,-14.75,13.25,2
2,-154.50,-5.00,1.50,-111.25,246.75,16.25,-21.75,5.00,2
3,-252.00,-32.00,-3.00,-115.75,248.50,-3.00,-22.00,-0.50,2
4,-424.50,-23.75,-18.00,-119.00,243.75,-13.50,-23.00,-0.50,2


In [6]:
df_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97133 entries, 0 to 97132
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       97133 non-null  float64
 1   1       97133 non-null  float64
 2   2       97133 non-null  float64
 3   3       97133 non-null  float64
 4   4       97133 non-null  float64
 5   5       97133 non-null  float64
 6   6       97133 non-null  float64
 7   7       97133 non-null  float64
 8   label   97133 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 6.7 MB


In [7]:
df_normal["label"].value_counts()

label
1    10000
9     9909
0     9830
3     9755
4     9664
5     9655
8     9641
2     9611
7     9604
6     9464
Name: count, dtype: int64

In [8]:
y = df_normal["label"]
x = df_normal.drop(columns = ["label"]) 

In [ ]:
x = abs(x)
x.head()

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=25)

In [18]:
from xgboost import XGBClassifier 
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import StratifiedKFold 

params = {
 'learning_rate': [0.1, 0.5, 0.8], 
'subsample': [0.6, 0.8, 1.0], 
'colsample_bytree': [0.6, 0.8, 1.0], 
'max_depth': [3, 4, 5] 
}

xgb = XGBClassifier(n_estimators=100, objective='multi:softmax', num_class=20,)

In [19]:
folds = 3 
strat_fold = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001) 
randomized_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=10, scoring = 'accuracy', n_jobs = 4, cv = strat_fold)

In [20]:
randomized_search.fit(x_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1001, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           import...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=100, n_jobs=None,
                                           num_class=20, num_parallel_tree=None, ...),
                   n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.1, 0.5, 0.8],
                                        'max_depth': [3, 4, 5],
                                        'subsample': [0.6, 0.8, 1.0]},
                   scoring='accuracy')

In [21]:
print('\n Best hyperparameters:') 
print(randomized_search.best_params_) 


 Best hyperparameters:
{'subsample': 1.0, 'max_depth': 5, 'learning_rate': 0.5, 'colsample_bytree': 0.8}


In [23]:
best_xgbclass = XGBClassifier(n_estimators=100, objective='multi:softmax', num_class=20,subsample= 1.0, max_depth= 5 , learning_rate= 0.5, colsample_bytree= 0.8) 
best_xgbclass.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=20,
              num_parallel_tree=None, ...)

In [24]:
 print(f"Training accuracy:{best_xgbclass.score(x_train, y_train)}\nTesting accuracy:{best_xgbclass.score(x_test, y_test)}")

Training accuracy:0.8786065426093224
Testing accuracy:0.7964688320378854
